## Day 47 Lecture 2 Assignment

In this assignment, we will perform K-Medoids clustering using responses to a survey about student life at a university.

In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd

from scipy.spatial.distance import pdist, squareform

# !pip3 install pyclustering
from pyclustering.cluster.kmedoids import kmedoids

import matplotlib.pyplot as plt

%matplotlib inline

<IPython.core.display.Javascript object>

This dataset consists of 35 binary features, each corresponding to a yes/no question that characterizes the student taking the survey.

This dataset contains a large number of features, each corresponding to a survey question. The feature name summarizes the survey question, so we will not list them all out here.

Load the dataset.

In [3]:
# answer goes here
url = "https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/student_life_survey.csv"
sl = pd.read_csv(url)
sl.head()

,Q2-Participated in Societies and Interest Groups,Q2-Participated in Clubs,"Q2-Participated in Halls, JCRCs and/or Residential College CSCs",Q2-Participated in University organised events,Q3-Interested in Arts & Culture,Q3-Interested in Science & Technology,Q3-Interested in Research and independent study,Q3-Interested in Sports,"Q3-Interested in Other competitions (eg case, debates)",Q3-Interested in Entrepreneurship,...,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others,response_id
0,0,1,0,0,0,1,1,0,1,0,...,1,0,0,0,0,0,0,0,0,1
1,0,1,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,2
2,0,0,1,0,0,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,3
3,1,1,1,1,0,1,1,0,0,0,...,1,0,1,1,1,1,0,1,0,4
4,1,0,1,1,0,1,1,0,0,1,...,1,1,0,1,0,0,0,1,0,5


<IPython.core.display.Javascript object>

For our analysis, we will focus on a specific subset of the survey that is focused on stress. These questions all begin with the string 'Q5'. Filter the columns that meet this criteria (should be 10 in total).

In [4]:
# answer goes here
sl_q5 = sl.filter(regex="Q5")
sl_q5 = sl_q5.sample(500)

<IPython.core.display.Javascript object>

The pyclustering implementation of kmedoids supports a variety of distance metrics, but they are primarily for numeric data. We will be using SMC/Hamming dissimilarity and precomputing the similarity matrix (an alternative would be to specify a user-defined function, which you are welcome to try in addition).

We'll assume for the next step that a pair of negative values (i.e. both responses are "no") is as informative as a pair of positive values. Compute the full distance/dissimilarity matrix for the survey data using matching/hamming distance.

In [5]:
# answer goes here
hamming_dist = pdist(sl_q5, metric="hamming")
hamming_dist = squareform(hamming_dist)
hamming_dist.shape

(500, 500)

<IPython.core.display.Javascript object>

Using the dissimilarity matrix, perform kmedoids clustering using k=2. Set the initial medoids randomly. Note that pyclustering expects the distance matrix to be a numpy array; a pandas dataframe may cause errors. 

Which survey responses are chosen as the cluster representatives? Print out the details of these responses.

In [6]:
# answer goes here
for i in range(5):
    init_medoids = np.random.randint(0, hamming_dist.shape[0], size=2)
    kmed = kmedoids(
        hamming_dist, initial_index_medoids=init_medoids, data_type="distance_matrix"
    )

    kmed.process()
    # print medoids to show how its changing
    print(kmed.get_medoids())

    # put in labels to show how its changing
    sl_q5[f"label_{i}"] = kmed.predict(hamming_dist)

[20, 187]
[20, 287]
[438, 370]
[497, 4]
[20, 85]


<IPython.core.display.Javascript object>

If you run the previous cell a few times, you'll probably notice that the medoids are very sensitive to initialization. A common approach to produce well-separated clusters is to choose initial centroids that are far apart. Re-run the previous process, except with a random pair of centroids that have a dissimilarity of 0.8 or higher. Are the results more stable now? How would you describe the typical clusters you see?

In [7]:
init_val = 0.8

for i in range(5):
    dissimiliar_idxs = np.where(hamming_dist > init_val)
    dissimiliar_pairs_df = pd.DataFrame(dissimiliar_idxs).T
    dissimiliar_pair = dissimiliar_pairs_df.sample(1).values[0]

    kmed = kmedoids(
        hamming_dist,
        initial_index_medoids=dissimiliar_pair,
        data_type="distance_matrix",
    )

    kmed.process()

    # print medoids to show how its changing
    print(kmed.get_medoids())

    sl_q5[f"dissimiliar_label_{i}"] = kmed.predict(hamming_dist)

[4, 40]
[33, 4]
[33, 4]
[286, 4]
[94, 4]


<IPython.core.display.Javascript object>